In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_prices = pd.read_csv('./data/item_prices.csv')
df_calendar = pd.read_csv('./data/daily_calendar_with_events.csv')

In [ ]:
df_prices.info()

In [ ]:
print(df_prices.shape)
df_prices.head(1)

## Drop duplicated

In [ ]:
df_prices[df_prices.duplicated(keep=False)]
df_prices.drop_duplicates(inplace=True)

## Delete nulls

In [ ]:
df_prices.isnull().sum()

In [ ]:
df_prices.dropna(inplace=True)

### I filter with groupby, for _items_

In [ ]:
df_new_price = df_prices.groupby(['item']).agg(
    price_min = ('sell_price', 'min'),
    price_max = ('sell_price', 'max'),
    price_mean = ('sell_price', 'mean')
).reset_index()
print(df_new_price.shape)
df_new_price.head(5)

In [ ]:
# file_name = "df_price_cluster.csv"
# df_new_price .to_csv(file_name)

In [ ]:
df_prices

### Create a new list of date

In [ ]:
start_date = df_calendar['date'].min()
end_date = df_calendar['date'].max()

In [ ]:
def create_yearweek_df(start_date_str, end_date_str, initial_week):
    #Convertir las cadenas de fecha a objetos datetime
    start_date = pd.to_datetime(start_date_str)
    end_date = pd.to_datetime(end_date_str)

    # Generar el rango de fechas
    date_range = pd.date_range(start_date, end_date)
    df = pd.DataFrame(date_range, columns=['date'])

    # Calcular el número de días desde la fecha de inicio
    df['dias_desde_inicio'] = (df['date'] - start_date).dt.days

    # Calcular el número de la semana ajustado desde la fecha inicial
    df['week_number'] = ((df['dias_desde_inicio'] // 7) + initial_week).astype(int)

    # Crear una columna 'yearweek' que combine el año y la semana
    df['year'] = df['date'].dt.year
    df['yearweek'] = df['year'].astype(str) + df['week_number'].apply(lambda x: f"{x:02d}")

    # Eliminar las columnas auxiliares
    df.drop(['dias_desde_inicio', 'week_number', 'year'], axis=1, inplace=True)

    return df

    # Crear DataFrames para cada año
df_2011 = create_yearweek_df('2011-01-29', '2011-12-30', 5)
df_2012_1 = create_yearweek_df('2011-12-31', '2012-01-01', 0)
df_2012 = create_yearweek_df('2012-01-02', '2012-12-30', 1)
df_2012_53 = pd.DataFrame([['2012-12-31','201253']],columns=['date','yearweek'])
df_2013 = create_yearweek_df('2013-01-01', '2013-12-31', 0)
df_2014 = create_yearweek_df('2014-01-01', '2014-12-31', 0)
df_2015 = create_yearweek_df('2015-01-01', '2015-12-31', 0)
df_2016_1 = create_yearweek_df('2016-01-01', '2016-01-02', 0)
df_2016 = create_yearweek_df('2016-01-03', '2016-04-24', 1)

df_2012_1.loc[0,'yearweek']='201152'

all_years_df = pd.concat([df_2011, df_2012_1, df_2012, df_2012_53, df_2013, df_2014, df_2015,df_2016_1, df_2016]).reset_index(drop=True)

In [ ]:
all_years_df['date']= pd.to_datetime(all_years_df['date'], format='%Y-%m-%d')

In [ ]:
all_years_df['yearweek'] = all_years_df['yearweek'].astype(str)

In [ ]:
df_prices['yearweek'] = df_prices['yearweek'].astype(str).apply(lambda x: x[:-2])

In [ ]:
df_prices.sample()

In [ ]:
df_merge = df_prices.merge(all_years_df, on=['yearweek'], how='left')

In [ ]:
df_merge.head()